In [1]:
import numpy as np
import os
from pandas import read_csv

from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.layers import Dropout, Merge
from keras.callbacks import ModelCheckpoint 
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import model_from_json
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.merge import add, concatenate

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.per_process_gpu_memory_fraction=0.3
session = tf.Session(config=config)

In [3]:
from config import csv_config as config
hdf5_path = config.HDF5_PATH
model_path = config.MODEL_PATH
weight_path = config.WEIGHT_PATH

In [4]:
seed = 7
np.random.seed(seed)

In [5]:
from helpers import CSVDatasetReader
reader = CSVDatasetReader(hdf5_path)
(Data, Labels) = reader.load()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(Data, Labels, test_size=0.33, random_state=seed)   
y_train = LabelBinarizer().fit_transform(y_train)
y_test = LabelBinarizer().fit_transform(y_test)

In [7]:
def model_wide():
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
def model_deep():
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def model_deep_wide():  
    wide_in = Input(shape=(8,))
    wide_out = Dense(1, input_dim=8, kernel_initializer='uniform', activation='sigmoid')(wide_in)
    wide = Model(wide_in, wide_out)

    deep_in = Input(shape=(8,))
    deep_out = Dense(8, input_dim=8, kernel_initializer='uniform', activation='relu')(deep_in)
    deep_out = Dense(4, kernel_initializer='uniform', activation='relu')(deep_out)
    deep_out = Dense(1, kernel_initializer='uniform', activation='sigmoid')(deep_out)
    deep = Model(deep_in, deep_out)
    
    concatenated = concatenate([wide_out, deep_out])
    out = Dense(1, activation='sigmoid')(concatenated)
    
    model = Model([wide_in, deep_in], out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
model_wide = model_wide()
model_wide.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                108       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model_wide.fit(X_train, y_train, validation_split=0.33, epochs=300, batch_size=10, verbose=0)
wide_score = model_wide.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model_wide.metrics_names[1], wide_score[1]*100))

acc: 76.38%


In [12]:
model_deep = model_deep()
model_deep.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_deep.fit(X_train, y_train, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
deep_score = model_deep.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model_deep.metrics_names[1], deep_score[1]*100))

acc: 77.56%


In [14]:
model_deep_wide = model_deep_wide()
model_deep_wide.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 8)            72          input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 4)            36          dense_7[0][0]                    
__________________________________________________________________________________________________
dense_6 (D

In [15]:
model_deep_wide.fit([X_train, X_train], y_train, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
deep_wide_score = model_deep_wide.evaluate([X_test, X_test], y_test, verbose=0)
print("%s: %.2f%%" % (model_deep_wide.metrics_names[1], deep_wide_score[1]*100))

acc: 77.95%


In [16]:
def model_deep_wide_merge():  
    wide = Sequential()
    wide.add(Dense(1, input_dim=8))

    deep = Sequential()
    deep.add(Dense(8, input_dim=8, kernel_initializer='uniform', activation='relu'))
    deep.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    deep.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

    model = Sequential()
    model.add(Merge([wide, deep], mode='concat', concat_axis=1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model_deep_wide_merge = model_deep_wide_merge()
model_deep_wide_merge.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 2)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 3         
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


C:\Users\boringtao\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()


In [18]:
model_deep_wide_merge.fit([X_train, X_train], y_train, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
deep_wide_merge_score = model_deep_wide_merge.evaluate([X_test, X_test], y_test, verbose=0)
print("%s: %.2f%%" % (model_deep_wide_merge.metrics_names[1], deep_wide_merge_score[1]*100))

acc: 74.80%


In [19]:
model_json = model_deep_wide.to_json()
with open(model_path + "baseline.json", "w") as json_file:
    json_file.write(model_json)
model_deep_wide_merge.save_weights(weight_path + "baseline.h5")

In [20]:
reader.close()